In [9]:
# Plot 3C waveforms of the whole array in a time window

In [10]:
## Import modules
from obspy import read, UTCDateTime, Stream
from matplotlib.pyplot import figure, subplots
import matplotlib.dates as mdates
from numpy import abs, mean, amin, amax
from pandas import read_csv, to_datetime
from os.path import join

from utils_basic import ROOTDIR, INNER_STATIONS, days_to_timestamps
from utils_preproc import preprocess_stream


In [11]:
## Inputs

subarray = "A"
starttime = "2020-01-13T13:02:00"
dur = 120.0 # Duration in seconds
plot_detect = True # Plot STA/LTA detections or not
sta_lta_dir = "detections_sta0.01s_lta0.20s_on5_off2"

freqmin = 10.0
freqmax = 450

scale = 1e-3 # Scale factor for plotting
size = 5 # Size of the dots for detections

In [12]:
## Read the station list
inpath = join(ROOTDIR, "stations.csv")
stadf = read_csv(inpath, index_col=0)
stadf = stadf.loc[stadf.subarray == subarray]
numst = len(stadf)

print(f"{numst} stations in subarray {subarray}")

19 stations in subarray A


In [13]:
## Read the STA/LTA detections
stalta_dict = {}    
if plot_detect:
    for station in stadf["name"]:
        inpath = join(ROOTDIR, sta_lta_dir, f"{station}.csv")
        detdf = read_csv(inpath, index_col=0, parse_dates=["trigger_time", "detrigger_time"])
        stalta_dict[station] = detdf

In [14]:
## Read and preprocess the data
starttime = UTCDateTime(starttime)
timewin = starttime.strftime("%Y-%m-%d-%H-00-00")
stream = Stream()
for station in stadf["name"]:
    try:
        pattern = join(ROOTDIR, timewin, f"*{station}*.SAC")
        stream += read(pattern)
    except:
        print(f"Could not read Station {station}")
        continue

stream = preprocess_stream(stream, freqmin, freqmax)
stream.trim(starttime, starttime + dur)

57 Trace(s) in Stream:

7F.A01..GH1 | 2020-01-13T13:02:00.000007Z - 2020-01-13T13:04:00.000015Z | 1000.0 Hz, 120001 samples
...
(55 other traces)
...
7F.A19..GHZ | 2020-01-13T13:02:00.000007Z - 2020-01-13T13:04:00.000015Z | 1000.0 Hz, 120001 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

In [21]:
fig, axes = subplots(nrows=1, ncols=3, sharex=True, sharey=True, figsize=(25, 1*numst))

for i, station in enumerate(stadf["name"]):

    ### Exract and plot the waveforms
    try:
        trace_z = stream.select(station=station, channel="*Z")[0]
        trace_1 = stream.select(station=station, channel="*1")[0]
        trace_2 = stream.select(station=station, channel="*2")[0]

        data_z = trace_z.data * scale + i
        data_1 = trace_1.data * scale + i
        data_2 = trace_2.data * scale + i
    except:
        print(f"Could not plot {station}")
        continue
    
    #### Convert the time axis to Pandas Timestamps
    timeax = trace_z.times("matplotlib")
    timeax = days_to_timestamps(timeax)


    axes[0].plot(timeax, data_z, color="black", linewidth=0.5)
    axes[1].plot(timeax, data_1, color="forestgreen", linewidth=0.5)
    axes[2].plot(timeax, data_2, color="royalblue", linewidth=0.5)

    axes[0].annotate(station, xy=(timeax[0], i+0.1), xycoords="data", fontsize=15, fontweight="bold")

    ### Extract and plot the detections
    if plot_detect:
        try:
            detdf = stalta_dict[station]
            detdf = detdf.loc[(detdf["trigger_time"] >= timeax[0]) & (detdf["trigger_time"] <= timeax[-1])]
        except:
            print(f"No detections for {station}")
            continue

        axes[0].scatter(detdf["trigger_time"], [i] * len(detdf), color="red", zorder=10)
        axes[1].scatter(detdf["trigger_time"], [i] * len(detdf), color="red", zorder=10)
        axes[2].scatter(detdf["trigger_time"], [i] * len(detdf), color="red", zorder=10)

axes[0].set_xlim([timeax[0], timeax[-1]])
axes[1].set_xlim([timeax[0], timeax[-1]])
axes[2].set_xlim([timeax[0], timeax[-1]])

# Format x-axis as date time
axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%dT%H:%M:%S'))
axes[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%dT%H:%M:%S'))
axes[2].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%dT%H:%M:%S'))

for label in axes[0].get_xticklabels():
    label.set_fontsize(15)
    label.set_verticalalignment('top')
    label.set_horizontalalignment('right')
    label.set_rotation(10)

for label in axes[1].get_xticklabels():
    label.set_fontsize(15)
    label.set_verticalalignment('top')
    label.set_horizontalalignment('right')
    label.set_rotation(10)

for label in axes[2].get_xticklabels():
    label.set_fontsize(15)
    label.set_verticalalignment('top')
    label.set_horizontalalignment('right')
    label.set_rotation(10)

# Set titles
axes[0].set_title("Up", fontsize=20, fontweight="bold")
axes[1].set_title("North", fontsize=20, fontweight="bold")
axes[2].set_title("East", fontsize=20, fontweight="bold")

# Set x label spacing to 60 seconds
axes[0].xaxis.set_major_locator(mdates.SecondLocator(interval=60))
axes[1].xaxis.set_major_locator(mdates.SecondLocator(interval=60))
axes[2].xaxis.set_major_locator(mdates.SecondLocator(interval=60))

# Turn off the y-axis labels and ticks
axes[0].set_yticks([])
axes[1].set_yticks([])
axes[2].set_yticks([])

axes[0].set_yticklabels([])
axes[1].set_yticklabels([])
axes[2].set_yticklabels([])

fig.tight_layout()
fig.patch.set_alpha(0.0)

## Save the figure
filename = f"sample_waveforms.png"

outpath = join(ROOTDIR, timewin, filename)
fig.savefig(outpath, dpi=300)
print(f"Saved to {outpath}")

In [16]:
stalta_dict

{'A01':                            trigger_time                   detrigger_time
 0      2020-01-10 11:00:00.198000+00:00 2020-01-10 11:00:00.286000+00:00
 1      2020-01-10 11:00:00.291000+00:00 2020-01-10 11:00:00.313000+00:00
 2      2020-01-10 11:00:00.342000+00:00 2020-01-10 11:00:00.367000+00:00
 3      2020-01-10 11:00:00.712000+00:00 2020-01-10 11:00:00.785000+00:00
 4      2020-01-10 11:00:00.808000+00:00 2020-01-10 11:00:00.824000+00:00
 ...                                 ...                              ...
 372403 2020-02-01 06:27:29.909101+00:00 2020-02-01 06:27:29.980101+00:00
 372404 2020-02-01 06:27:30.414101+00:00 2020-02-01 06:27:30.481101+00:00
 372405 2020-02-01 06:27:30.822101+00:00 2020-02-01 06:27:30.879101+00:00
 372406 2020-02-01 06:27:31.323101+00:00 2020-02-01 06:27:31.342101+00:00
 372407 2020-02-01 06:27:31.592101+00:00 2020-02-01 06:27:31.615101+00:00
 
 [372408 rows x 2 columns],
 'A02':                            trigger_time                   detrigger